In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from laspy.file import File

import sys
sys.path.insert(0,'..')
from point_density_functions import *

In [2]:
# Load nyc dataframe
nyc_file_dir = '../../Data/NYC_topo/'
nyc_pt_file = ['las_points_NYC_975172.lz']
nyc_df = pd.read_hdf(nyc_file_dir+nyc_pt_file[0])

#Sort by gps_time
nyc_df.sort_values(by=['gps_time'],inplace=True)

In [ ]:
plt.hist(nyc_df['scan_angle'][:10000000])

In [3]:
def label_returns(las_df):
    '''
    Parses the flag_byte into number of returns and return number, adds these fields to las_df.
    Input - las_df - dataframe from .laz or .lz file
    Output - first_return_df - only the first return points from las_df.
           - las_df - input dataframe with num_returns and return_num fields added 
    '''
    
    las_df['num_returns'] = np.floor(las_df['flag_byte']/16).astype(int)
    las_df['return_num'] = las_df['flag_byte']%16
    first_return_df = las_df[las_df['return_num']==1]
    first_return_df = first_return_df.reset_index(drop=True)
    return first_return_df, las_df
# Note: this anlaysis doesn't really need this
_,nyc_df = label_returns(nyc_df)

In [4]:
gap_times = np.array(nyc_df.iloc[1:]['gps_time']) - np.array(nyc_df.iloc[:-1]['gps_time'])

In [5]:
# Largest time gap in days
# Topographic vs. Bathymetric

gap_times.max()/(60*60*24)

53.93011730718944

In [6]:
nyc_df.columns

Index(['X', 'Y', 'Z', 'intensity', 'flag_byte', 'classification_flags',
       'classification_byte', 'user_data', 'scan_angle', 'pt_src_id',
       'gps_time', 'x_scaled', 'y_scaled', 'z_scaled', 'time_gap',
       'num_returns', 'return_num'],
      dtype='object')

In [7]:
# Gaps between flights
gap_times[gap_times>30]

array([7.09614133e+02, 1.79690707e+02, 6.69055595e+02, 1.87644041e+02,
       6.62764546e+02, 4.65956214e+06, 3.60791444e+02, 3.35617147e+02,
       3.43717758e+02, 3.26562820e+02, 3.60826608e+02, 4.38196903e+02,
       3.61490675e+02, 2.08452254e+02, 3.89018295e+02, 1.90233535e+03])

In [8]:
nyc_df['scan_type'] = ['topographic']*nyc_df.shape[0]
nyc_df.loc[5818507:]['scan_type'] = ['bathymetric']*nyc_df.loc[5818507:].shape[0]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
# Add flight_id for nyc flights based on time gaps
nyc_df['flight_id'] = [0]*nyc_df.shape[0]
flight_id = 0
old_i = 0
for i,gap in enumerate(gap_times):
    if gap > 30:
        nyc_df.loc[old_i:i+1,'flight_id'] = flight_id
        flight_id+=1
        old_i = i
nyc_df.loc[old_i:,'flight_id'] = flight_id

In [ ]:
# Create hdf file
nyc_df.to_hdf(nyc_file_dir+'las_points_nyc_flight_id',key='df',complevel=1,complib='lzo')

In [ ]:
nyc_df.sort_values(by=['gps_time'],inplace=True)
nyc_df.reset

In [ ]:
nyc_df['scan_type']

In [ ]:
nn = File('../../Data/NYC_topo/975172.las',mode='r')

In [ ]:
nyc_df.reset_index(inplace=True)
for i in range(105068,1000000):
    if abs(nyc_df['scan_angle'][i+1] - nyc_df['scan_angle'][i]) > 3000:
        print(nyc_df.iloc[i])

In [ ]:
plt.plot(nyc_df['scan_angle'][:1000000])

In [ ]:
pt1 = nyc_df.iloc[a]
pt2 = nyc_df.iloc[a+1]
dist = np.sqrt((pt1['x_scaled']-pt2['x_scaled'])**2+(pt1['y_scaled']-pt2['y_scaled'])**2)
dist/3.28

In [4]:
nyc_df.head()

,X,Y,Z,intensity,flag_byte,classification_flags,classification_byte,user_data,scan_angle,pt_src_id,gps_time,x_scaled,y_scaled,z_scaled,time_gap,num_returns,return_num
34185,97668434,17499927,650,16280,17,92,1,1,2272,1192,1.789515e+08,976684.34,174999.27,6.5,0.000000,1,1
34186,97668827,17499847,610,31181,17,28,1,1,2270,1192,1.789515e+08,976688.27,174998.47,6.1,0.000006,1,1
34187,97668766,17499690,640,25110,17,28,1,1,2272,1192,1.789515e+08,976687.66,174996.90,6.4,0.000006,1,1
34188,97668808,17499620,640,22627,17,92,1,1,2274,1192,1.789515e+08,976688.08,174996.20,6.4,0.000006,1,1
34189,97668876,17499797,620,27870,17,92,1,1,2271,1192,1.789515e+08,976688.76,174997.97,6.2,0.000006,1,1
